## This notebook demonstrates how to use the ghost module implemented in Mirage.
### The baseline scene is the focus field (NAP-10) with dominated by many bright stars, which is consequently dominated by ghosts
(But here we only simulate subset of the sources to save time.)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import yaml
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
from glob import glob
import jwst

from astropy.io import fits,ascii
from astropy.modeling import models, fitting
from astropy.table import QTable, Table, Column


In [ ]:
# Deconstruct;
from NIRCAM_Gsim.grism_seed_disperser import Grism_seed
from mirage.seed_image import catalog_seed_image
from mirage.yaml import yaml_generator
from mirage import imaging_simulator
from mirage import wfss_simulator
from mirage.ramp_generator import obs_generator
from mirage.dark import dark_prep


In [ ]:
jwst.__version__

In [ ]:
import mirage
mirage.__version__

## 1. Create yaml files based on APT output files
If users can provide a set of yaml files, then you can skip this part.

In [ ]:
# This is input directory;
DIR_INP = './files/'

# This is output directory;
DIR_OUT = './output/'
if not os.path.exists(DIR_OUT):
    os.mkdir(DIR_OUT)


In [ ]:
# Get yaml file;
# Those should be retrieved from APT.
xml_file      = './files/1085.xml' 
pointing_file = './files/1085.pointing'

# Set output directory for Yaml;
yaml_output_dir = './yaml/'
if not os.path.exists(yaml_output_dir):
    os.mkdir(yaml_output_dir)

    
# Setup misc.:

## Source catalog;
file_point_name = './files/stars_example.txt' 
catalogs = {'point_source': file_point_name}

reffile_defaults = 'crds'
cosmic_rays = {'library': 'SUNMAX', 'scale': 1.0}
dates = '2022-07-01'

pav3 = 0
background = 'low'
simulations_output_dir = 'output'
datatype = 'raw'


# Run the yaml generator

yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                              catalogs=catalogs, cosmic_rays=cosmic_rays,
                              background=background,roll_angle=pav3,
                              dates=dates, reffile_defaults=reffile_defaults,
                              verbose=True, output_dir=yaml_output_dir,
                              simdata_output_dir=simulations_output_dir,
                              datatype=datatype,add_ghosts=True)

yam.create_inputs()

## 2. Run simulator

In [ ]:
pfiles = glob('%s/*01101_*_nis.yaml'%yaml_output_dir)
pfiles

In [ ]:
yaml_WFSS_files = []
yaml_NIRISS_imaging_files = []

for f in pfiles:
    my_dict = yaml.safe_load(open(f))
    if my_dict["Inst"]["mode"]=="wfss":
        yaml_WFSS_files.append(f)
    if my_dict["Inst"]["mode"]=="imaging" and my_dict['Inst']['instrument']=='NIRISS':
        yaml_NIRISS_imaging_files.append(f)
    
print("WFSS files:",len(yaml_WFSS_files))
print("Imaging files:",len(yaml_NIRISS_imaging_files))

In [ ]:
yaml_NIRISS_imaging_files

In [ ]:
# Direct images;

for file in yaml_NIRISS_imaging_files[:1]:
    img_sim = imaging_simulator.ImgSim()
    img_sim.paramfile = file
    img_sim.create()
    

In [ ]:
# WFSS;

for file in yaml_WFSS_files[:]:
    m = wfss_simulator.WFSSSim(file, override_dark=None, save_dispersed_seed=True,
                               extrapolate_SED=None, disp_seed_filename=None, source_stamps_file=None,
                               SED_file=None, SED_normalizing_catalog_column=None, SED_dict=None,
                               create_continuum_seds=True)
    
    m.create()


### Done!